In [39]:
# import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.io as sio
# from sideblock_distances import DistanceSensorsVessel
from ac_res_anlys_funcs import pickTOF

from bokeh.plotting import figure, show, save
from bokeh.io import output_notebook, output_file, reset_output
from bokeh.layouts import gridplot, row, column
output_notebook()

Loading BokehJS ...

In [211]:
SAVE = 0
runname = 'p5483'
run = 'run1'

T = 1
R = 1

In [212]:
# load acoustic data (output from pXXXX_a.m)
[Amp, AmpRef, Filter, FreqQ, LocalAcTime, MaxInter, 
NZPad, NtoStack, RmsAmp, RmsAmpRef, TimeRange, TimeShift, 
freqQAmp, fullWFref, idx2analyze, maxAmp, maxFreq, ref, threshold] = list(map(lambda bb: np.load('/Users/clay/Documents/research/frac-hydroOsc/mat_ac_analysis/Results_'+runname+'_'+run+'_fullrun_Stack3WFs_absref_wAmp.npz', allow_pickle=True)[bb],
                                                                              ['Amp', 'AmpRef', 'Filter', 'FreqQ', 'LocalAcTime', 'MaxInter','NZPad', 'NtoStack', 'RmsAmp', 'RmsAmpRef', 'TimeRange', 'TimeShift','freqQAmp', 
                                                                               'fullWFref', 'idx2analyze', 'maxAmp', 'maxFreq', 'ref', 'threshold']))

# load acoustic settings mat file
[acSettings, numCHR, numCHT, Nsamples, fs] = list(map(lambda cc: np.load('../mat_ac_analysis/'+runname+'_'+run+'.npz', allow_pickle=True)[cc], ['acSettings', 'numCHR', 'numCHT', 'Nsamples', 'fs']))

timeWF = pickTOF(idx2analyze, T, R, Nsamples, fs, fullWFref, runname, run, SAVE=0)

Loading BokehJS ...

In [213]:
fullWFref[0:100,R,T] = 0 # zero electrical noise that precedes all WFs

AIC = np.zeros(len(fullWFref[:,R,T]))

for j in range(len(fullWFref[:,R,T])):
    AIC[j] = j*np.log10(np.var(fullWFref[0:j,R,T]))+(len(fullWFref[:,R,T])-j-1)*np.log10(np.var(fullWFref[j+1:len(fullWFref[:,R,T]),R,T]));

AIC[AIC == -np.inf] = np.nanmax(AIC);
AIC[len(fullWFref[:,R,T])-1] = np.nanmax(AIC)

AIC_min = np.nanmin(AIC)
AIC_max = np.nanmax(AIC)
AIC_idx = AIC == AIC_min

<ipython-input-213-2709afa039ae>:6: RuntimeWarning: divide by zero encountered in log10
  AIC[j] = j*np.log10(np.var(fullWFref[0:j,R,T]))+(len(fullWFref[:,R,T])-j-1)*np.log10(np.var(fullWFref[j+1:len(fullWFref[:,R,T]),R,T]));


In [217]:
from bokeh.models import LinearAxis, Range1d, Span

fig2 = figure(plot_width=800, plot_height=400, tools='pan,box_zoom,undo,hover,crosshair') 
fig2.line(timeWF, fullWFref[:,R,T], line_width=2, line_color="indigo")

fig2.extra_y_ranges = {"foo": Range1d(start=AIC_min*0.995, end=AIC_max*1.005)}
fig2.add_layout(LinearAxis(y_range_name="foo"), 'right')
fig2.line(timeWF, AIC, line_width=2, line_color="fuchsia", y_range_name="foo")

fig2.circle(timeWF[AIC_idx], AIC_min, size=7, color="blue", y_range_name="foo")
fig2.circle(timeWF[AIC_idx], fullWFref[AIC_idx,R,T], size=7, color="blue")

vline = Span(location=timeWF[AIC_idx][0], dimension='height', line_color='blue', line_width=2)
fig2.renderers.extend([vline])

fig2.yaxis.axis_label = 'Reference WF (.)'
fig2.xaxis.axis_label = 'Time (us)'
show(fig2)

In [210]:
# from scipy import signal as sigs

# orderFIR = 256
# f=0.1
# b, a = sigs.butter(516, 0.1, 'high', analog=True)

# y = sigs.filtfilt(b, a, fullWFref[:,R,T], padlen=1)



# fig2 = figure(plot_width=800, plot_height=400, tools='pan,box_zoom,undo,hover,crosshair') 
# fig2.line(timeWF, fullWFref[:,R,T], line_width=2, line_color="indigo")

# # fig2.extra_y_ranges = {"foo": Range1d(start=AIC_min*0.995, end=AIC_max*1.005)}
# # fig2.add_layout(LinearAxis(y_range_name="foo"), 'right')
# fig2.line(timeWF, y, line_width=2, line_color="fuchsia") #, y_range_name="foo"

# # fig2.circle(timeWF[AIC_idx], AIC_min, size=7, color="fuchsia", y_range_name="foo")
# # fig2.circle(timeWF[AIC_idx], fullWFref[AIC_idx,R,T], size=7, color="fuchsia")

# # vline = Span(location=timeWF[AIC_idx][0], dimension='height', line_color='fuchsia', line_width=2)
# # fig2.renderers.extend([vline])

# fig2.yaxis.axis_label = 'Reference WF (.)'
# fig2.xaxis.axis_label = 'Time (us)'
# show(fig2)

In [157]:
# ps = np.abs(np.fft.fft(fullWFref[:,R,T]))**2

# time_step = 1 / 30
# freqs = np.fft.fftfreq(fullWFref[:,R,T].size, fs)
# idx = np.argsort(freqs)

# # plt.plot(freqs[idx], ps[idx])


# fig2 = figure(plot_width=800, plot_height=400, tools='pan,box_zoom,undo,hover,crosshair') 
# fig2.line(freqs[idx], ps[idx], line_width=2, line_color="black")

# # fig2.extra_y_ranges = {"foo": Range1d(start=AIC_min*0.995, end=AIC_max*1.005)}
# # fig2.add_layout(LinearAxis(y_range_name="foo"), 'right')
# # fig2.line(timeWF, y, line_width=2, line_color="fuchsia") #, y_range_name="foo"

# # fig2.circle(timeWF[AIC_idx], AIC_min, size=7, color="fuchsia", y_range_name="foo")
# # fig2.circle(timeWF[AIC_idx], fullWFref[AIC_idx,R,T], size=7, color="fuchsia")

# # vline = Span(location=timeWF[AIC_idx][0], dimension='height', line_color='fuchsia', line_width=2)
# # fig2.renderers.extend([vline])

# fig2.yaxis.axis_label = 'Reference WF (.)'
# fig2.xaxis.axis_label = 'Time (us)'
# show(fig2)

In [159]:
f, Pxx_den = sigs.welch(fullWFref[:,R,T], fs, nperseg=1024)

# f_med, Pxx_den_med = signal.welch(, fs, nperseg=1024, average='median')

# plt.semilogy(f, Pxx_den, label='mean')

fig2 = figure(plot_width=800, plot_height=400, tools='pan,box_zoom,undo,hover,crosshair') 
fig2.line(f, Pxx_den, line_width=2, line_color="black")
fig2.yaxis.axis_label = 'freq (Hz)'
fig2.xaxis.axis_label = 'Time (us)'
show(fig2)

In [ ]:
# # picking = 'AIC';

# B = fir1(256,0.1,'low');

# for i = 8:8
    
#     waveform = p4976_run1_ACdata_test(2048*(i-1)+1:2048*i);
    
#     output = filtfilt(B,1,waveform);
    
#     output(1:120) = 0;

#     plot(waveform + 3000*(i-1)); 
#     hold on; plot(output + 3000*(i-1));


# #  dcmObj = datacursormode;
# #  set(dcmObj,'UpdateFcn',@GoodCursor);
 
#  AIC = zeros(1,length(waveform));
 
# #  switch picking
     
#      case {'AIC'}
     
#          for j=1:(length(output)-1)
             
#                  AIC(j) = j.*log10(var(output(1:j)))+(length(output)-j-1).*log10(var(output(j+1:length(output))));
         
#          end
         
#          AIC(AIC == -Inf) = max(AIC); 
#          AIC(length(output)) = max(AIC);

#          [~, AIC_idx]= min(AIC);

#          plot(AIC + 3000*(i-1));  
#          plot(AIC_idx, output(AIC_idx)+3000*(i-1), 'k*');

#      end

 
# end